# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

In [3]:
len(CORD19_CSV['doi'])

77448

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

In [5]:
#delete Nans
doi_counted = doi.value_counts()
doi_counted

10.1016/j.dsx.2020.04.012       2
10.31729/jnma.5498              2
10.1007/978-0-387-73637-2_3     1
10.1038/s41433-020-01130-4      1
10.3758/s13421-020-01129-y      1
                               ..
10.1101/2020.08.06.20169896     1
10.3390/ijerph17124552          1
10.1111/eci.13433               1
10.1371/journal.pone.0042972    1
10.1016/j.ajem.2020.04.052      1
Name: doi, Length: 74302, dtype: int64

In [6]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [7]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [8]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
35345,"[{'affiliation-city': 'Bangkok', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '2',..."
35346,None,None
35347,"{'affiliation-city': 'Taipei', 'affilname': 'N...","{'srctype': 'j', 'eid': '2-s2.0-2142809650', '..."
35348,"{'affiliation-city': 'Hong Kong', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-85094137791', ..."


In [9]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

35350

In [10]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 35350 -> 10.1057/s41311-020-00258-0
Position: 35351 -> 10.1038/s41598-020-77794-5
Position: 35352 -> 10.1016/j.jaging.2020.100892
Position: 35353 -> 10.1007/s00705-016-3208-x
Position: 35354 -> 10.3390/bios9040117
Position: 35355 -> 10.1093/infdis/jiaa274
Position: 35356 -> 10.3892/ijmm.2014.1830
Position: 35357 -> 10.1016/j.annemergmed.2020.06.004
Position: 35358 -> 10.1016/j.patter.2020.100018
Position: 35359 -> 10.1016/j.jinf.2020.02.014
Position: 35360 -> 10.34171/mjiri.34.97
Position: 35361 -> 10.1016/j.aohep.2020.10.001
Position: 35362 -> 10.1177/0020764020940742
Position: 35363 -> 10.1101/2020.08.07.241653
Position: 35364 -> 10.1007/s11606-020-05878-z
Position: 35365 -> 10.1038/s41598-020-78467-z
Position: 35366 -> 10.1007/s11538-020-00778-z
Position: 35367 -> 10.3389/fped.2020.559389
Position: 35368 -> 10.1007/s40961-020-00219-1
Position: 35369 -> 10.1016/j.jpha.2020.04.008
Position: 35370 -> 10.3389/fcimb.2020.00470
Position: 35371 -> 10.1016/j.jsat.2020.108163
Posit

Position: 35533 -> 10.3390/ijerph17197180
Position: 35534 -> 10.1007/978-1-4614-0266-4_37
Position: 35535 -> 10.1016/j.epidem.2021.100437
Position: 35536 -> 10.1016/j.molimm.2019.06.005
Position: 35537 -> 10.1007/978-3-030-49435-3_20
Position: 35538 -> 10.1017/ice.2020.401
Position: 35539 -> 10.3390/v12101174
Position: 35540 -> 10.1055/a-1164-6611
Position: 35541 -> 10.1371/journal.ppat.1004374
Position: 35542 -> 10.1186/s12959-020-00231-0
Position: 35543 -> 10.1371/journal.pone.0213674
Position: 35544 -> 10.1183/23120541.00562-2020
Position: 35545 -> 10.1007/s12204-020-2169-0
Position: 35546 -> 10.1016/j.jpag.2020.08.007
Position: 35547 -> 10.1101/2020.10.25.20219170
Position: 35548 -> 10.1101/2020.10.29.20222000
Position: 35549 -> 10.1016/j.micpath.2019.05.010
Position: 35550 -> 10.1007/s40615-020-00942-0
Position: 35551 -> 10.7717/peerj.9879
Position: 35552 -> 10.1177/2382120520959695
Position: 35553 -> 10.1016/j.dld.2020.05.017
Position: 35554 -> 10.1186/s12912-021-00539-w
Position

Position: 35714 -> 10.1038/s41431-020-0629-5
Position: 35715 -> 10.1016/j.amjms.2020.09.017
Position: 35716 -> 10.1007/978-981-13-3390-3_5
Position: 35717 -> 10.3390/geriatrics5040088
Position: 35718 -> 10.1101/2020.08.10.241869
Position: 35719 -> 10.1186/s12859-019-2878-2
Position: 35720 -> 10.1186/s12931-019-1181-0
Position: 35721 -> 10.4061/2011/128318
Position: 35722 -> 10.1101/2020.06.28.20141929
Position: 35723 -> 10.1007/s10488-021-01113-x
Position: 35724 -> 10.1101/2020.06.01.20119560
Position: 35725 -> 10.1016/j.virusres.2013.06.019
Position: 35726 -> 10.1101/2020.04.10.036343
Position: 35727 -> 10.1016/j.scitotenv.2020.143493
Position: 35728 -> 10.1007/s40521-020-00275-7
Position: 35729 -> 10.1016/j.vetmic.2014.03.017
Position: 35730 -> 10.1186/s13063-020-04584-9
Position: 35731 -> 10.1136/bmjopen-2020-037138
Position: 35732 -> 10.1007/s10815-020-01952-x
Position: 35733 -> 10.1177/0020764020941567
Position: 35734 -> 10.1101/2021.01.12.20249080
Position: 35735 -> 10.1371/journ

Position: 35896 -> 10.1002/pdi.2306
Position: 35897 -> 10.1007/s10956-020-09855-3
Position: 35898 -> 10.3390/genes11091002
Position: 35899 -> 10.1111/j.1863-2378.2012.01539.x
Position: 35900 -> 10.15585/mmwr.mm6943e5
Position: 35901 -> 10.1016/j.rvsc.2018.09.002
Position: 35902 -> 10.1016/b978-0-12-416002-6.00021-3
Position: 35903 -> 10.1016/j.watres.2009.02.002
Position: 35904 -> 10.1101/2020.07.23.217372
Position: 35905 -> 10.1093/oxrep/graa033
Position: 35906 -> 10.1007/s00330-020-07252-3
Position: 35907 -> 10.1183/13993003.01727-2020
Position: 35908 -> 10.1101/2021.02.01.21250973
Position: 35909 -> 10.1016/j.jpolmod.2020.03.014
Position: 35910 -> 10.1017/s0950268820002150
Position: 35911 -> 10.3390/molecules25194433
Position: 35912 -> 10.1016/j.jim.2006.12.003
Position: 35913 -> 10.1038/s41598-019-57283-0
Position: 35914 -> 10.7717/peerj.9914
Position: 35915 -> 10.1007/978-1-84800-925-7_9
Position: 35916 -> 10.1101/2020.03.17.995209
Position: 35917 -> 10.1101/2020.08.06.240333
Posi

Position: 36078 -> 10.1101/2020.04.19.048710
Position: 36079 -> 10.1007/s13149-013-0318-5
Position: 36080 -> 10.1007/s12273-019-0513-9
Position: 36081 -> 10.1016/j.prevetmed.2019.104816
Position: 36082 -> 10.1016/j.tracli.2020.08.009
Position: 36083 -> 10.1371/journal.pone.0243299
Position: 36084 -> 10.1016/j.psep.2020.10.048
Position: 36085 -> 10.1016/j.matlet.2020.128819
Position: 36086 -> 10.1007/s00287-020-01285-0
Position: 36087 -> 10.1371/journal.pone.0241954
Position: 36088 -> 10.1016/j.psychres.2020.113469
Position: 36089 -> 10.6061/clinics/2020/e2217
Position: 36090 -> 10.1101/2020.04.14.20065524
Position: 36091 -> 10.1101/2020.03.27.009480
Position: 36092 -> 10.1136/bmjopen-2020-039849
Position: 36093 -> 10.1371/journal.pone.0162061
Position: 36094 -> 10.1016/j.ejphar.2020.173741
Position: 36095 -> 10.1038/s41372-019-0489-4
Position: 36096 -> 10.1016/j.eng.2020.06.009
Position: 36097 -> 10.1016/j.genhosppsych.2020.08.013
Position: 36098 -> 10.1093/ehjcr/ytaa188
Position: 3609

...

Position: 40813 -> 10.1016/j.diabres.2020.108463
Position: 40814 -> 10.1007/s11096-020-01017-0
Position: 40815 -> 10.1016/j.buildenv.2020.107368
Position: 40816 -> 10.1016/j.jtbi.2020.110536
Position: 40817 -> 10.1186/s12879-017-2784-z
Position: 40818 -> 10.1101/2020.08.20.258772
Position: 40819 -> 10.1016/j.acci.2020.09.005
Position: 40820 -> 10.1016/j.lfs.2020.118469
Position: 40821 -> 10.1007/s12024-018-0043-2
Position: 40822 -> 10.1186/s13018-020-01976-0
Position: 40823 -> 10.1016/j.jiac.2020.10.026
Position: 40824 -> 10.1177/0269216320940566
Position: 40825 -> 10.4102/jamba.v12i1.1010
Position: 40826 -> 10.1111/dme.14374
Position: 40827 -> 10.1534/g3.118.200185
Position: 40828 -> 10.1016/j.jvoice.2020.12.042
Position: 40829 -> 10.1007/s42399-020-00592-7
Position: 40830 -> 10.3390/foods9101480
Position: 40831 -> 10.1007/978-3-030-50423-6_5
Position: 40832 -> 10.1186/s41469-020-00074-z
Position: 40833 -> 10.1101/2020.06.18.20135004
Position: 40834 -> 10.1007/s12529-010-9083-z
Positi

Position: 40995 -> 10.1371/journal.pone.0159196
Position: 40996 -> 10.3201/eid2701.203399
Position: 40997 -> 10.1016/j.jmoldx.2020.09.004
Position: 40998 -> 10.1016/j.eap.2020.09.017
Position: 40999 -> 10.1136/bmjhci-2020-100209
Position: 41000 -> 10.2196/17154
Position: 41001 -> 10.1371/journal.ppat.1004447
Position: 41002 -> 10.1007/978-3-030-49418-6_9
Position: 41003 -> 10.1080/07391102.2020.1825232
Position: 41004 -> 10.1016/j.healthplace.2020.102404
Position: 41005 -> 10.1007/s00705-006-0840-x
Position: 41006 -> 10.3389/fbioe.2018.00011
Position: 41007 -> 10.1016/j.meegid.2020.104648
Position: 41008 -> 10.3390/jcm8020192
Position: 41009 -> 10.3390/biomedicines8120561
Position: 41010 -> 10.1186/1471-2105-9-s1-s23
Position: 41011 -> 10.1016/j.annonc.2020.08.1707
Position: 41012 -> 10.1007/s00251-016-0940-z
Position: 41013 -> 10.1186/1471-2334-9-196
Position: 41014 -> 10.1006/viro.1996.0413
Position: 41015 -> 10.3390/nu12113346
Position: 41016 -> 10.1101/2020.02.29.971101
Position: 4

Position: 41177 -> 10.3390/molecules25215079
Position: 41178 -> 10.1016/j.cca.2020.05.027
Position: 41179 -> 10.1007/s13042-019-01029-x
Position: 41180 -> 10.1016/j.cities.2020.102904
Position: 41181 -> 10.3390/ijerph17114056
Position: 41182 -> 10.1186/s12864-019-6016-3
Position: 41183 -> 10.1007/s12603-021-1598-2
Position: 41184 -> 10.1007/s11250-020-02439-8
Position: 41185 -> 10.1016/j.mjafi.2020.04.012
Position: 41186 -> 10.1016/j.jvb.2020.103481
Position: 41187 -> 10.1371/journal.ppat.1000648
Position: 41188 -> 10.1016/j.ijcha.2021.100715
Position: 41189 -> 10.1016/j.promfg.2020.05.122
Position: 41190 -> 10.1016/s2173-5115(08)70297-5
Position: 41191 -> 10.30476/ijcbnm.2020.86478.1347
Position: 41192 -> 10.1016/j.ijtb.2020.09.032
Position: 41193 -> 10.1093/brain/awr268
Position: 41194 -> 10.1007/s11695-020-04883-9
Position: 41195 -> 10.1093/asjof/ojaa037
Position: 41196 -> 10.1007/s10654-019-00595-2
Position: 41197 -> 10.1007/978-3-030-50436-6_41
Position: 41198 -> 10.1101/2021.02.0

Position: 41359 -> 10.1016/j.childyouth.2020.105784
Position: 41360 -> 10.1016/j.jadohealth.2020.09.039
Position: 41361 -> 10.1007/s42770-020-00398-8
Position: 41362 -> 10.3390/medicina56060289
Position: 41363 -> 10.1007/s11886-020-01393-z
Position: 41364 -> 10.1186/s40478-020-00960-3
Position: 41365 -> 10.3390/nu12123764
Position: 41366 -> 10.3389/fpubh.2020.00340
Position: 41367 -> 10.1136/bmjopen-2020-039897
Position: 41368 -> 10.3389/fmicb.2019.00319
Position: 41369 -> 10.1053/j.ro.2020.03.001
Position: 41370 -> 10.1016/j.matpr.2020.10.055
Position: 41371 -> 10.1007/s43450-020-00002-y
Position: 41372 -> 10.3389/fgene.2020.00641
Position: 41373 -> 10.1101/2020.09.15.298547
Position: 41374 -> 10.3390/ijerph17239080
Position: 41375 -> 10.1016/j.wjorl.2021.01.001
Position: 41376 -> 10.1101/2021.01.29.21250643
Position: 41377 -> 10.1371/journal.pone.0240286
Position: 41378 -> 10.1002/jcp.29868
Position: 41379 -> 10.3390/healthcare8040515
Position: 41380 -> 10.1093/ofid/ofaa528
Position:

KeyboardInterrupt: 

In [11]:
dict_new_extra_info

{35350: {'affiliation': {'affiliation-city': 'Canterbury',
   'affilname': 'University of Kent',
   'affiliation-country': 'United Kingdom'},
  'coredata': {'srctype': 'j',
   'prism:issueIdentifier': '3',
   'eid': '2-s2.0-85087943255',
   'prism:coverDate': '2021-06-01',
   'prism:aggregationType': 'Journal',
   'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85087943255',
   'subtypeDescription': 'Article',
   'dc:creator': {'author': [{'ce:given-name': 'Richard',
      'preferred-name': {'ce:given-name': 'Richard',
       'ce:initials': 'R.',
       'ce:surname': 'Sakwa',
       'ce:indexed-name': 'Sakwa R.'},
      '@seq': '1',
      'ce:initials': 'R.',
      '@_fa': 'true',
      'affiliation': {'@id': '60010818',
       '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60010818'},
      'ce:surname': 'Sakwa',
      '@auid': '6506132065',
      'author-url': 'https://api.elsevier.com/content/author/author_id/6506132065',
      'ce:indexed-na

In [21]:
df_new_extra_info = pd.DataFrame(dict_new_extra_info)

In [22]:
df_new_extra_info

,35350,35351,35352,35353,35354,35355,35356,35357,35358,35359,...,41408,41409,41410,41411,41412,41413,41414,41415,41416,41417
affiliation,"{'affiliation-city': 'Canterbury', 'affilname'...","[{'affiliation-city': 'Rehovot', 'affilname': ...","[{'affiliation-city': 'Vancouver', 'affilname'...","[{'affiliation-city': 'Seoul', 'affilname': 'S...","[{'affiliation-city': 'Waltham', 'affilname': ...","{'affiliation-city': 'Columbus', 'affilname': ...","{'affiliation-city': 'Changchun', 'affilname':...",NaN,"{'affiliation-city': 'Boston', 'affilname': 'H...","[{'affiliation-city': 'Xi'an', 'affilname': 'X...",...,None,"{'affiliation-city': 'Shanghai', 'affilname': ...","[{'affiliation-city': 'Badr', 'affilname': 'Eg...","[{'affiliation-city': 'Guelph', 'affilname': '...","[{'affiliation-city': 'Shatin', 'affilname': '...","[{'affiliation-city': 'Perth', 'affilname': 'M...","[{'affiliation-city': 'Guangzhou', 'affilname'...","[{'affiliation-city': 'Sydney', 'affilname': '...","[{'affiliation-city': 'Hangzhou', 'affilname':...","[{'affiliation-city': 'Singapore City', 'affil..."
coredata,"{'srctype': 'j', 'prism:issueIdentifier': '3',...","{'srctype': 'j', 'prism:issueIdentifier': '1',...","{'srctype': 'j', 'eid': '2-s2.0-85092928639', ...","{'srctype': 'j', 'eid': '2-s2.0-85010754995', ...","{'srctype': 'j', 'prism:issueIdentifier': '4',...","{'srctype': 'j', 'eid': '2-s2.0-85087433403', ...","{'srctype': 'j', 'eid': '2-s2.0-84905057923', ...","{'srctype': 'j', 'eid': '2-s2.0-85092393829', ...","{'srctype': 'j', 'prism:issueIdentifier': '1',...","{'srctype': 'j', 'eid': '2-s2.0-85081742934', ...",...,None,"{'srctype': 'j', 'eid': '2-s2.0-84864356122', ...","{'srctype': 'j', 'eid': '2-s2.0-85097910130', ...","{'srctype': 'j', 'eid': '2-s2.0-67649976592', ...","{'srctype': 'j', 'prism:issueIdentifier': '1',...","{'srctype': 'j', 'prism:issueIdentifier': '11'...","{'srctype': 'j', 'eid': '2-s2.0-85096874998', ...","{'srctype': 'j', 'prism:issueIdentifier': '1',...","{'srctype': 'j', 'prism:issueIdentifier': '1',...","{'srctype': 'j', 'eid': '2-s2.0-40949146126', ..."


In [23]:
df_new_extra_transposed = df_new_extra_info.T
df_new_extra_transposed

,affiliation,coredata
35350,"{'affiliation-city': 'Canterbury', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '3',..."
35351,"[{'affiliation-city': 'Rehovot', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
35352,"[{'affiliation-city': 'Vancouver', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85092928639', ..."
35353,"[{'affiliation-city': 'Seoul', 'affilname': 'S...","{'srctype': 'j', 'eid': '2-s2.0-85010754995', ..."
35354,"[{'affiliation-city': 'Waltham', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
...,...,...
41413,"[{'affiliation-city': 'Perth', 'affilname': 'M...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
41414,"[{'affiliation-city': 'Guangzhou', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85096874998', ..."
41415,"[{'affiliation-city': 'Sydney', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."
41416,"[{'affiliation-city': 'Hangzhou', 'affilname':...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."


In [24]:
df_combined_extra_info = pd.concat([df_current_extra_info, df_new_extra_transposed],ignore_index=True)

In [17]:
df_combined_extra_info.to_pickle('extra_info_CS5099.pkl')

In [18]:
entry = AffiliationsFromScopusByDOI(client, '10.1086/605034')
entry

{'affiliation': [{'affiliation-city': 'Chiba',
   'affilname': 'Chiba University',
   'affiliation-country': 'Japan'},
  {'affiliation-city': 'Hanoi',
   'affilname': 'National Hospital of Pediatrics Hanoi',
   'affiliation-country': 'Viet Nam'},
  {'affiliation-city': 'Tokyo',
   'affilname': 'National Institute of Infectious Diseases',
   'affiliation-country': 'Japan'},
  {'affiliation-city': 'Tokyo',
   'affilname': 'National Center for Global Health and Medicine',
   'affiliation-country': 'Japan'},
  {'affiliation-city': 'Miyazaki',
   'affilname': 'University of Miyazaki Faculty of Medicine',
   'affiliation-country': 'Japan'},
  {'affiliation-city': 'Tokyo',
   'affilname': 'Tokai University',
   'affiliation-country': 'Japan'}],
 'coredata': {'srctype': 'j',
  'prism:issueIdentifier': '4',
  'eid': '2-s2.0-69149099607',
  'pubmed-id': '19591579',
  'prism:coverDate': '2009-08-15',
  'prism:aggregationType': 'Journal',
  'prism:url': 'https://api.elsevier.com/content/abstract/s

In [19]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

{
    "affiliation": [
        {
            "affiliation-city": "Chiba",
            "affilname": "Chiba University",
            "affiliation-country": "Japan"
        },
        {
            "affiliation-city": "Hanoi",
            "affilname": "National Hospital of Pediatrics Hanoi",
            "affiliation-country": "Viet Nam"
        },
        {
            "affiliation-city": "Tokyo",
            "affilname": "National Institute of Infectious Diseases",
            "affiliation-country": "Japan"
        },
        {
            "affiliation-city": "Tokyo",
            "affilname": "National Center for Global Health and Medicine",
            "affiliation-country": "Japan"
        },
        {
            "affiliation-city": "Miyazaki",
            "affilname": "University of Miyazaki Faculty of Medicine",
            "affiliation-country": "Japan"
        },
        {
            "affiliation-city": "Tokyo",
            "affilname": "Tokai University",
            "affiliatio

In [20]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

,affiliation-city,affilname,affiliation-country
0,Chiba,Chiba University,Japan
1,Hanoi,National Hospital of Pediatrics Hanoi,Viet Nam
2,Tokyo,National Institute of Infectious Diseases,Japan
3,Tokyo,National Center for Global Health and Medicine,Japan
4,Miyazaki,University of Miyazaki Faculty of Medicine,Japan
5,Tokyo,Tokai University,Japan


In [ ]:
entry['coredata']

In [ ]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")
doi_doc